# Data cleaning and analysis

## Summary of data analysis and findings

In [1]:
import os
import constants
import df_utils
import pandas as pd
import seaborn as sns
from pprint import pprint
import matplotlib.pyplot as plt


In [2]:
# Load data into Pandas DataFrame
data_path = "assignment-sample-data.csv"
df = pd.read_csv(data_path)

In [3]:
from IPython.display import display
pd.set_option('display.width', 1000)

In [4]:
# Show first entries
print(df.head())
#print(df.tail())

        parking_id   area_type   parking_start_time     parking_end_time  parking_fee currency   parkinguser_id           car_id        lat        lon account_type
0  fake_c28a323810  SurfaceLot  2015-03-06 19:55:41  2015-03-06 20:07:00         8.50      SEK  fake_bf5d9b530e  fake_130ae2aeb1  59.246370  18.077019    corporate
1  fake_76c21cf355  SurfaceLot  2015-03-06 18:08:20  2015-03-06 19:46:00        15.67      SEK  fake_bf5d9b530e  fake_130ae2aeb1  59.231789  18.083995    corporate
2  fake_995ed971a6    OnStreet  2017-07-21 09:55:42  2017-07-21 14:23:50        67.00      SEK  fake_3ba346a0cd  fake_f7a9d564d9  59.350331  18.096649    corporate
3  fake_6b81ea4f35  SurfaceLot  2017-07-24 07:21:12  2017-07-24 07:34:31         4.34      SEK  fake_ea19a50003  fake_fae7e31b34  59.315826  18.098355    corporate
4  fake_424b61e0eb  SurfaceLot  2015-03-09 12:05:46  2015-03-09 13:57:54        50.50      SEK  fake_1cc1970582  fake_0755f3c71f  59.320919  18.047513    corporate


- 11 columns
- I suspect that that is sorted by account_type since first 50 are corporate and last 50 are private. This means we should shuffle before training

In [5]:
shuffle_rows = True
if shuffle_rows:
    df = df.sample(frac=1).reset_index(drop=True)

In [6]:
print(df.columns)

Index(['parking_id', 'area_type', 'parking_start_time', 'parking_end_time', 'parking_fee', 'currency', 'parkinguser_id', 'car_id', 'lat', 'lon', 'account_type'], dtype='object')


In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   parking_id          87489 non-null  object 
 1   area_type           87489 non-null  object 
 2   parking_start_time  87489 non-null  object 
 3   parking_end_time    87489 non-null  object 
 4   parking_fee         87489 non-null  float64
 5   currency            87489 non-null  object 
 6   parkinguser_id      87489 non-null  object 
 7   car_id              87489 non-null  object 
 8   lat                 87489 non-null  float64
 9   lon                 87489 non-null  float64
 10  account_type        87489 non-null  object 
dtypes: float64(3), object(8)
memory usage: 7.3+ MB
None


- 11 columns from raw dataset
- 87489 rows (transactions)
- No null values
- Mix of float (3) and object (8) dtypes

In [8]:
print(df.describe())

        parking_fee           lat           lon
count  87489.000000  87489.000000  87489.000000
mean      31.786153     58.786174     17.035432
std       39.338515      2.925073      2.639340
min        0.000000   -180.006219   -180.006783
25%        9.000000     59.292461     17.137638
50%       18.750000     59.332389     18.000587
75%       39.100000     59.360741     18.066440
max      983.000000     67.871132     54.009971


- Exists some outliers for parking_fee
    - min = 0 (how does this make sense?)
    - max = 983 (this is very high compared to 75th percentile that is 39)
    - mean = 31
    - std = 39 (quite high compared to mean => high data variance)
    - Could for example remove outliers and standardize the data for easier training
- Latidute and longitude values are not in the correct range
    - min lat = -90 (but we observe -180)
    - offsetting wont help since the range is above 180, meaning I do not trust its values to be correctly measured
- Longitude is however in the correct range [-180, 180]
- Latitude and longitude data by itself does not really give much information and would require some kind of embedding model inbetween, that maps the coordinate point to some feature space that includes information about the location and parking behaviour

In [9]:
print(df["account_type"].unique())
print(df["account_type"].value_counts())
print(5*"-")
print(df["currency"].unique())
print(df["currency"].value_counts())
print(5*"-")
print(df["area_type"].unique())
print(df["area_type"].value_counts())
print(5*"-")
print(f"Number of rows in df: {len(df)}")
print(f"Number of unique parking_ids: {df['parking_id'].nunique()}")
print(f"Number of unique parkinguser_ids: {df['parkinguser_id'].nunique()}")
print(f"Number of unique car_ids: {df['car_id'].nunique()}")

['private' 'corporate']
account_type
private      57239
corporate    30250
Name: count, dtype: int64
-----
['SEK' 'NOK' 'EUR' 'DKK']
currency
SEK    86918
NOK      465
DKK       87
EUR       19
Name: count, dtype: int64
-----
['OnStreet' 'SurfaceLot' 'AboveGroundGarage' 'Administrative'
 'UndergroundGarage' 'EVC' 'CameraParkArea']
area_type
OnStreet             52747
SurfaceLot           26749
Administrative        6269
UndergroundGarage     1248
AboveGroundGarage      459
EVC                     14
CameraParkArea           3
Name: count, dtype: int64
-----
Number of rows in df: 87489
Number of unique parking_ids: 87489
Number of unique parkinguser_ids: 300
Number of unique car_ids: 1652


- not an exact 50/50 split between private and corporate transactions, but they are in the same order of magnitude so stratified sampling is not super necessary
- There are 4 different currencies used in the dataset, huge majority is in SEK. 2 options which are both viable
    - convert all to SEK equivalent
    - remove transactions in other currencies
- area_type have 7 different values, "OnStreet" and "SurfaceLot" are the most common, while "EVC" and "CameraParkArea" are the least common
    - Before removing "EVC" and "CameraParkArea", we should check if there are any patterns (correlation) in the data that we could use to predict the account_type from those samples
- area_type would require categorical encoding, before being used as input to the model.
- parking_id is unique as it is a primary key and equal to number of rows in the dataset, meaning it is completely useless for the model
- There are a total of 300 registered private and corporate parking users (accounts)
- There are a total of 1652 unique car ids used for parking transactions. Since this is larger than the number of parking users, some users have multiple cars, which is indicate of a business account (business has a fleet of cars). Could be useful to create a feature of number of used/owned cars per user

### Check if transactions with "parkinguser_id" exits with multiple account types (corporate and private)

In [10]:
parkinguser_ids = {} # parkinguser_id -> account_type

# Loop over all transactions
for index, row in df.iterrows(): 
    user_id = row['parkinguser_id']
    account_type = row['account_type']

    if user_id not in parkinguser_ids: # If user_id is not in the dictionary, add it
        parkinguser_ids[user_id] = account_type
    else: # If user_id is already in the dictionary, check if the account_type is the same

        if parkinguser_ids[user_id] != account_type:
            print(f"User {user_id} has multiple account types: {parkinguser_ids[user_id]} and {account_type}")

- Each parkinguser_id has only one account_type. If a person has both a private and a corporate account, then they make transactions using different parkinguser_ids!

### Max datetime range

In [11]:
# Data time range
df['parking_start_time'] = pd.to_datetime(df["parking_start_time"])
df['parking_end_time'] = pd.to_datetime(df["parking_end_time"])

# Find the earliest start time and latest end time
earliest_start = df['parking_start_time'].min()
latest_end = df['parking_end_time'].max()

# Calculate the total time span
time_span = latest_end - earliest_start

print(f"Earliest transaction start: {earliest_start}")
print(f"Latest transaction end: {latest_end}")
print(f"Total time span: {time_span.days} days, {time_span.seconds // 3600} hours")

Earliest transaction start: 2013-02-08 10:40:58
Latest transaction end: 2020-09-30 14:34:35
Total time span: 2791 days, 3 hours


- The time span of the dataset is very long (7 years). So normalizing with this time span is not a good idea. Some users could have been active for a long time while some could have been active for a short time.
- Better to normalize with the time span of each parkinguser_id

## Data cleaning

In [12]:
# remove duplicate rows
# remove outliers in some features
# remove nonsensical values (parking fee = 0 ?)
# convert date columns to datetime

## Feature Engineering

### Currency conversion

In [13]:
df["parking_fee_sek"] = df[["currency", "parking_fee"]].apply(df_utils.convert_currency, axis=1)
print(df[df["currency"] == "EUR"].head())

           parking_id       area_type  parking_start_time    parking_end_time  parking_fee currency   parkinguser_id           car_id        lat        lon account_type  parking_fee_sek
1119  fake_b6a291edfe        OnStreet 2019-01-02 14:51:02 2019-01-02 18:21:29         3.50      EUR  fake_ade3f3f432  fake_e996210230  46.532441  12.132596      private          38.8850
2078  fake_56f5699a58      SurfaceLot 2019-01-05 16:37:22 2019-01-05 17:30:00         2.00      EUR  fake_ade3f3f432  fake_e996210230  46.463216  12.204619      private          22.2200
3836  fake_336bec8abf        OnStreet 2018-06-05 15:13:50 2018-06-05 15:30:00         1.05      EUR  fake_7047370fa5  fake_1fbe5a7dbe  60.397872  25.669604      private          11.6655
4832  fake_b8b80c5daf  Administrative 2016-04-07 11:09:54 2016-04-07 12:38:00         3.21      EUR  fake_f78d5f4769  fake_33a47b6e0a  52.370624   9.731980      private          35.6631
8398  fake_adc2bd373e        OnStreet 2019-08-06 07:20:04 2019-08-06 1

### Parking duration

In [17]:
df["parking_duration"] = df[["parking_start_time", "parking_end_time"]].apply(lambda x: (x[1] - x[0]).total_seconds() / 3600, axis=1)

/tmp/ipykernel_17095/1278808334.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df["parking_duration"] = df[["parking_start_time", "parking_end_time"]].apply(lambda x: (x[1] - x[0]).total_seconds() / 3600, axis=1)


### Weekday calculation



In [18]:
df["weekday"] = df.apply(df_utils.get_weekday, axis=1)

### Unique cars used per account

In [19]:
# Count unique cars per user
user_car_counts = df.groupby("parkinguser_id")["car_id"].nunique()

# Map the counts back to the original dataframe
df["registered_cars"] = df["parkinguser_id"].map(user_car_counts)

# Display the first few rows to verify
print(df[["parking_id","parkinguser_id", "car_id", "registered_cars", "account_type"]].head(10))

        parking_id   parkinguser_id           car_id  registered_cars account_type
0  fake_f7707765d9  fake_ade3f3f432  fake_644ff1469a               12      private
1  fake_6572f13c7f  fake_7f7916179f  fake_31f2599e4b                7      private
2  fake_568ed9edae  fake_2c5fc36fef  fake_a38ca99970                1    corporate
3  fake_a69ceb5cdb  fake_ade3f3f432  fake_644ff1469a               12      private
4  fake_cc13a48b34  fake_c28849a708  fake_38be7eaaa7               22    corporate
5  fake_c60479976c  fake_64ca8b3b64  fake_642cbbea17                3    corporate
6  fake_132feae99e  fake_92164957dc  fake_b52aeacb25                3      private
7  fake_ed3200fadd  fake_e1ea8dfecc  fake_837cc49b6f               16    corporate
8  fake_da5a8c042c  fake_d418b2f74a  fake_1d83d00ba0                6      private
9  fake_1cfcad93f2  fake_835dae552c  fake_a0034300b7                8    corporate


### Parking count

In [20]:
# Count total parking transactions per user
user_parking_counts = df['parkinguser_id'].value_counts()
df['n_parkings'] = df['parkinguser_id'].map(user_parking_counts)
print(df[['parkinguser_id', 'n_parkings', "account_type"]].head(10))

    parkinguser_id  n_parkings account_type
0  fake_ade3f3f432         998      private
1  fake_7f7916179f         298      private
2  fake_2c5fc36fef         327    corporate
3  fake_ade3f3f432         998      private
4  fake_c28849a708        2053    corporate
5  fake_64ca8b3b64         132    corporate
6  fake_92164957dc         120      private
7  fake_e1ea8dfecc         837    corporate
8  fake_d418b2f74a         304      private
9  fake_835dae552c         328    corporate


### Parking activity (normalized w.r.t days)

In [21]:
# Normalize parking frequency with respect to days used


# Ensure the timestamp columns are in datetime format
df["parking_start_time"] = pd.to_datetime(df["parking_start_time"])
df["parking_end_time"] = pd.to_datetime(df["parking_end_time"])

# Calculate account age in days for each user
user_first_last = df.groupby("parkinguser_id").agg(
    first_parking=("parking_start_time", "min"),
    last_parking=("parking_end_time", "max")
)

# Calculate account age in days (add 1 to avoid division by zero for single transactions)
user_first_last["account_age_days"] = (user_first_last["last_parking"] - user_first_last["first_parking"]).dt.days + 1

# Get total parkings per user (we already have this in "n_parkings")
# Calculate parking activity (parkings per day)
user_first_last["parking_activity"] = df.groupby("parkinguser_id").size() / user_first_last["account_age_days"]

# Map the parking_activity back to the original dataframe
df = df.merge(
    user_first_last[["parking_activity"]],
    left_on="parkinguser_id",
    right_index=True,
    how="left"
)

# Display the results
print("Sample of user parking activity (parkings per day):")
print(df[["parkinguser_id", "n_parkings", "parking_activity", "account_type"]].head(10))
print("\nSummary statistics for parking_activity:")
print(df["parking_activity"].describe())

Sample of user parking activity (parkings per day):
    parkinguser_id  n_parkings  parking_activity account_type
0  fake_ade3f3f432         998          0.563205      private
1  fake_7f7916179f         298          0.210452      private
2  fake_2c5fc36fef         327          1.097315    corporate
3  fake_ade3f3f432         998          0.563205      private
4  fake_c28849a708        2053          1.154668    corporate
5  fake_64ca8b3b64         132          0.081835    corporate
6  fake_92164957dc         120          0.071942      private
7  fake_e1ea8dfecc         837          0.417248    corporate
8  fake_d418b2f74a         304          0.130528      private
9  fake_835dae552c         328          0.181015    corporate

Summary statistics for parking_activity:
count    87489.000000
mean         0.493419
std          0.344318
min          0.006500
25%          0.210452
50%          0.392388
75%          0.762215
max          1.333543
Name: parking_activity, dtype: float64


- This feature has pros and cons. If the parking activity is uniform then it will measure average activity pretty well. However, if the activity is not uniform (such as a big break between parkings) then the acitivity will be underestimated.
- Parking activity could be a good way to measure activity per day is a better measure compared to total parking transactions since a private user that has used the app for a long time will have more parking transactions than a new business user that has just started using the app.

## Aggregate over user

## Save processed data

In [22]:
save_df = False
if save_df:
    clean_path = data_path[:-4] + '-cleaned.csv'
    df.to_csv(clean_path, index=False)
